In [48]:
import os
import requests
from collections import OrderedDict

In [14]:
YOUR_API_KEY = '1234'
data_file_path = r"C:\Users\blumb\main\Jupyter\APIs\data\dests.txt"

In [16]:
with open(data_file_path) as file:
    destinations = file.read().splitlines()


In [17]:
print(destinations)

['Istanbul', 'Amsterdam', 'Valletta', 'Basel', 'Doha']


In [18]:
#txt 
destinationsStr = "|".join(destinations)
print(destinationsStr)

Istanbul|Amsterdam|Valletta|Basel|Doha


In [19]:
origin = "Jerusalem"

In [20]:
url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin}&destinations={destinationsStr}&key={YOUR_API_KEY}"
payload={}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload)
distanceMatrixjsonResponse = response.json()

In [21]:
url = f"https://maps.googleapis.com/maps/api/geocode/json?address={destinationsStr}&key={YOUR_API_KEY}"
payload={}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload)
geocodejsonResponse = response.json()

In [22]:
print(geocodejsonResponse)

{'results': [{'address_components': [{'long_name': 'Amsterdam', 'short_name': 'Amsterdam', 'types': ['locality', 'political']}, {'long_name': 'Government of Amsterdam', 'short_name': 'Government of Amsterdam', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'North Holland', 'short_name': 'NH', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Netherlands', 'short_name': 'NL', 'types': ['country', 'political']}], 'formatted_address': 'Amsterdam, Netherlands', 'geometry': {'bounds': {'northeast': {'lat': 52.4311573, 'lng': 5.0791619}, 'southwest': {'lat': 52.278174, 'lng': 4.7287589}}, 'location': {'lat': 52.3675734, 'lng': 4.9041389}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 52.4311573, 'lng': 5.0791619}, 'southwest': {'lat': 52.278174, 'lng': 4.7287589}}}, 'partial_match': True, 'place_id': 'ChIJVXealLU_xkcRja_At0z9AGY', 'types': ['locality', 'political']}, {'address_components': [{'long_name': 'İstanbul', 'short_name'

In [23]:
geocodeResults = {}
for address_components_index, address_components in enumerate(geocodejsonResponse['results']):
    geocodeResults[destinations[address_components_index]] = {'lat': address_components['geometry']['location']['lat'],
                                                              'lng': address_components['geometry']['location']['lng']}

In [26]:
print(repr(geocodeResults))

{'Istanbul': {'lat': 52.3675734, 'lng': 4.9041389}, 'Amsterdam': {'lat': 41.0082376, 'lng': 28.9783589}, 'Valletta': {'lat': 25.2854473, 'lng': 51.53103979999999}, 'Basel': {'lat': 35.8989085, 'lng': 14.5145528}, 'Doha': {'lat': 47.5595986, 'lng': 7.5885761}}


In [28]:
resultsList = distanceMatrixjsonResponse["rows"][0]['elements']

In [41]:
resultsDictionary = {}
for row_index, row in enumerate(resultsList):
    resultsDictionary[destinations[row_index]] = (row['distance']['value'],
                                                  row['duration']['value']/60/60,
                                                  row['duration']['value']/60,
                                                  geocodeResults[destinations[row_index]]['lng'],
                                                  geocodeResults[destinations[row_index]]['lat'])

In [42]:
print(repr(resultsDictionary))

{'Istanbul': (1808887, 20.64361111111111, 1238.6166666666666, 4.9041389, 52.3675734), 'Amsterdam': (4527771, 47.48166666666667, 2848.9, 28.9783589, 41.0082376), 'Valletta': (3782994, 50.489444444444445, 3029.366666666667, 51.53103979999999, 25.2854473), 'Basel': (4091368, 43.64805555555555, 2618.883333333333, 14.5145528, 35.8989085), 'Doha': (2104502, 21.973055555555558, 1318.3833333333334, 7.5885761, 47.5595986)}


In [43]:
for row_index, row in enumerate(resultsList):
    print(f"Destination: {destinations[row_index]}")
    print(f"\tDistance from Jerusalem[KM]: {row['distance']['value']}")
    print(f"\tArrival duration from Jerusalem[Hrs]: {row['duration']['value']/60/60}")
    print(f"\tArrival duration from Jerusalem[Minutes]: {row['duration']['value']/60}")
    print(f"\tLongitude: {geocodeResults[destinations[row_index]]['lng']}")
    print(f"\tLatitude: {geocodeResults[destinations[row_index]]['lat']}")

Destination: Istanbul
	Distance from Jerusalem[KM]: 1808887
	Arrival duration from Jerusalem[Hrs]: 20.64361111111111
	Arrival duration from Jerusalem[Minutes]: 1238.6166666666666
	Longitude: 4.9041389
	Latitude: 52.3675734
Destination: Amsterdam
	Distance from Jerusalem[KM]: 4527771
	Arrival duration from Jerusalem[Hrs]: 47.48166666666667
	Arrival duration from Jerusalem[Minutes]: 2848.9
	Longitude: 28.9783589
	Latitude: 41.0082376
Destination: Valletta
	Distance from Jerusalem[KM]: 3782994
	Arrival duration from Jerusalem[Hrs]: 50.489444444444445
	Arrival duration from Jerusalem[Minutes]: 3029.366666666667
	Longitude: 51.53103979999999
	Latitude: 25.2854473
Destination: Basel
	Distance from Jerusalem[KM]: 4091368
	Arrival duration from Jerusalem[Hrs]: 43.64805555555555
	Arrival duration from Jerusalem[Minutes]: 2618.883333333333
	Longitude: 14.5145528
	Latitude: 35.8989085
Destination: Doha
	Distance from Jerusalem[KM]: 2104502
	Arrival duration from Jerusalem[Hrs]: 21.973055555555558

In [70]:
def find_three_furthest(resultsList):
    """Arguments:
       resultsList:  The destination dictionary
       
       Retruns:
           Three furthest destinations from Jerusalem."""
    distance_dict = {}
    for row_index, row in enumerate(resultsList):
        distance_dict[destinations[row_index]] = (row['distance']['value'])
    #print(repr(distance_dict))
    sorted_distance_dict = sorted(distance_dict, key=lambda i: int(distance_dict[i]), reverse=True)
    print(repr(sorted_distance_dict))
    return sorted_distance_dict

In [71]:
furthest_destination = find_three_furthest(resultsList)

['Amsterdam', 'Basel', 'Valletta', 'Doha', 'Istanbul']


In [76]:
three_furthest_destination = furthest_destination[0:3]

In [77]:
print(three_furthest_destination)

['Amsterdam', 'Basel', 'Valletta']
